In [38]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [39]:
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold
from tensorflow.keras.layers import Dense, Dropout 


In [40]:
df= pd.read_csv('Student_Performance_Mat.csv')

In [41]:
df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,11,12,10


In [42]:
#convert categorical values to numerical values 
label1=LabelEncoder()
df["school"]=label1.fit_transform(df["school"])
df["sex"]=label1.fit_transform(df["sex"])
df["address"]=label1.fit_transform(df["address"])
df["famsize"]=label1.fit_transform(df["famsize"])
df["Pstatus"]=label1.fit_transform(df["Pstatus"])
df["Medu"]=label1.fit_transform(df["Medu"])
df["Mjob"]=label1.fit_transform(df["Mjob"])
df["Fjob"]=label1.fit_transform(df["Fjob"])
df["reason"]=label1.fit_transform(df["reason"])
df["guardian"]=label1.fit_transform(df["guardian"])
df["schoolsup"]=label1.fit_transform(df["schoolsup"])
df["famsup"]=label1.fit_transform(df["famsup"])
df["paid"]=label1.fit_transform(df["paid"])
df["activities"]=label1.fit_transform(df["activities"])
df["nursery"]=label1.fit_transform(df["nursery"])
df["higher"]=label1.fit_transform(df["higher"])
df["internet"]=label1.fit_transform(df["internet"])
df["romantic"]=label1.fit_transform(df["romantic"])
df['G1'] = df['G1'].map(lambda x: str(x).replace('""','')).astype('float')
df['G2'] = df['G2'].map(lambda x: str(x).replace('""','')).astype('float')
df['G3'] = df['G3'].map(lambda x: str(x).replace('"','')).astype('float')
df=pd.get_dummies(df, drop_first=True, columns=["Mjob"])
df=pd.get_dummies(df, drop_first=True, columns=["Fjob"])
df=pd.get_dummies(df, drop_first=True, columns=["reason"])
df=pd.get_dummies(df, drop_first=True, columns=["guardian"])

In [43]:
df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,...,Mjob_4,Fjob_1,Fjob_2,Fjob_3,Fjob_4,reason_1,reason_2,reason_3,guardian_1,guardian_2
0,0,0,18,1,0,0,4,4,2,2,...,0,0,0,0,1,0,0,0,1,0
1,0,0,17,1,0,1,1,1,1,2,...,0,0,1,0,0,0,0,0,0,0
2,0,0,15,1,1,1,1,1,1,2,...,0,0,1,0,0,0,1,0,1,0
3,0,0,15,1,0,1,4,2,1,3,...,0,0,0,1,0,1,0,0,1,0
4,0,0,16,1,0,1,3,3,1,2,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,1,1,20,1,1,0,2,2,1,2,...,0,0,0,1,0,0,0,0,0,1
391,1,1,17,1,1,1,3,1,2,1,...,0,0,0,1,0,0,0,0,1,0
392,1,1,21,0,0,1,1,1,1,1,...,0,0,1,0,0,0,0,0,0,1
393,1,1,18,0,1,1,3,2,3,1,...,0,0,1,0,0,0,0,0,1,0


In [44]:
Y = df.iloc[:,28].values #identify label 

In [45]:
df = df.drop(['G3'],axis=1)

In [46]:
#identify features 
X = df.iloc[:,:41].values 
#Scale data
sc = StandardScaler()
X = sc.fit_transform(X)
 


In [47]:
#Splitting data in Training 60% / Validation 20% / Test 20&
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)


In [48]:
#Scale data
scaler=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [49]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import LearningRateScheduler


In [52]:
# KFold classification
kf = KFold(10, shuffle=True, random_state=42) 
    
oos_y = []
oos_pred = []

fold = 0
for train, test in kf.split(X):
    fold+=1
    print(f"Fold #{fold}")
    
    x_train = X[train]
    y_train = Y[train]
    x_test = X[test]
    y_test = Y[test]
    
    
    model = Sequential()
    model.add(Dense(41, activation = 'relu', input_dim = 41))
    model.add(Dense(units = 41, activation = 'relu'))
    model.add(Dense(units = 41, activation = 'relu'))
    model.add(Dense(units = 41, activation = 'relu'))
    #model.add(Dense(units = 1, activation = 'linear'))
    model.add(Dense(units = 1))
    sgd = tf.keras.optimizers.SGD(learning_rate=0.0001)
    model.compile(loss='mean_squared_error', optimizer='sgd')

    model.fit(x_train,y_train,validation_data=(x_test,y_test),verbose=0,epochs=100)
   

    
    pred = model.predict(x_test)
    
    oos_y.append(y_test)
    oos_pred.append(pred)    

    # Measure this fold's RMSE
    score = np.sqrt(metrics.mean_squared_error(pred,y_test))
    print(f"Fold score (RMSE): {score}")
    

# Build the oos prediction list and calculate the error.
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
score = np.sqrt(metrics.mean_squared_error(oos_pred,oos_y))
print(f"Final, out of sample score (RMSE): {score}")    




Fold #1
Fold score (RMSE): 3.132591233323626
Fold #2
Fold score (RMSE): 2.558752899326155
Fold #3
Fold score (RMSE): 2.4875813624132124
Fold #4
Fold score (RMSE): 2.724667008641092
Fold #5
Fold score (RMSE): 2.2421346299809786
Fold #6
Fold score (RMSE): 1.7817546807540319
Fold #7
Fold score (RMSE): 2.5195518109273367
Fold #8
Fold score (RMSE): 2.130409201113963
Fold #9
Fold score (RMSE): 2.547616130427637
Fold #10
Fold score (RMSE): 1.9157955322558298
Final, out of sample score (RMSE): 2.436346856639167
